# 트리 (Tree)

`-` 사이클이 없는 연결 그래프

## 트리의 부모 찾기

- 문제 출처: [백준 11725번](https://www.acmicpc.net/problem/11725)

`-` 각 노드에 대해 모든 연결 노드를 그래프에 기록한다

`-` 루트 노드부터 BFS를 사용하자

`-` 탐색 과정에서 다음 노드의 부모는 현재 노드가 된다

In [1]:
from collections import defaultdict, deque


def solution():
    N = int(input())
    ROOT = 1
    graph = defaultdict(list)
    visited = {ROOT: True}
    node2parent = {}
    for _ in range(N - 1):
        a, b = map(int, input().split())
        graph[a].append(b)
        graph[b].append(a)
    queue = deque([ROOT])
    while queue:
        node = queue.popleft()
        for next_node in graph[node]:
            if next_node not in visited:
                visited[next_node] = True
                node2parent[next_node] = node
                queue.append(next_node)
    for node in range(2, N + 1):
        print(node2parent[node])


solution()

# input
# 12
# 1 2
# 1 3
# 2 4
# 3 5
# 3 6
# 4 7
# 4 8
# 5 9
# 5 10
# 6 11
# 6 12

 12
 1 2
 1 3
 2 4
 3 5
 3 6
 4 7
 4 8
 5 9
 5 10
 6 11
 6 12


1
1
2
3
3
4
4
5
5
6
6


## 트리 순회

- 문제 출처: [백준 1991번](https://www.acmicpc.net/problem/1991)

`-` 전형적인 트리 순회 문제

`-` 클래스와 재귀 함수를 이용해 깔끔하게 구현할 수 있다

`-` 내 코드보다 좋은 코드가 많이 있음 (배우자)

In [2]:
from collections import defaultdict


def preorder_traversal(tree, start, visited={}):
    visited[start] = True
    for next_node in tree[start]:
        if next_node != ABSENCE and next_node not in visited:
            preorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def inorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited):
        visited[start] = True
    for next_node in [left, start, right]:
        if next_node != ABSENCE and next_node not in visited:
            inorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def postorder_traversal(tree, start, visited={}):
    left = tree[start][0]
    right = tree[start][1]
    if (left == ABSENCE or left in visited) and (right == ABSENCE or right in visited):
        visited[start] = True
    for next_node in [left, right, start]:
        if next_node != ABSENCE and next_node not in visited:
            postorder_traversal(tree, next_node, visited)
    return list(visited.keys())


def solution():
    global ABSENCE
    N = int(input())
    tree = defaultdict(list)
    ROOT = "A"
    ABSENCE = "."
    for _ in range(N):
        parent, left, right = input().split()
        tree[parent].append(left)
        tree[parent].append(right)
    preorder_result = preorder_traversal(tree, ROOT)
    inorder_result = inorder_traversal(tree, ROOT)
    postorder_result = postorder_traversal(tree, ROOT)
    for result in [preorder_result, inorder_result, postorder_result]:
        print("".join(result))


solution()

# input
# 7
# A B C
# B D .
# C E F
# E . .
# F . G
# D . .
# G . .

 7
 A B C
 B D .
 C E F
 E . .
 F . G
 D . .
 G . .


ABDCEFG
DBAECFG
DBEGFCA


## 트리의 지름

- 문제 출처: [백준 19647번](https://www.acmicpc.net/problem/1967)

`-` [$\star$] `주의 사항: 이진 트리가 아닐 수도 있다` [$\star$]

`-` 트리의 지름이 되는 양 끝 노드를 생각해보면 둘은 자식 노드를 가질 수 없다

`-` $s$와 $t$가 트리의 지름이 된다고 해보자

`-` 만약 $s$에게 자식 노드 $u$가 있다면 $u\to s \to t$가 기존 트리의 지름보다 더 길다

`-` 이는 $s$와 $t$가 트리의 지름이 된다는 가정과 모순된다

`-` 따라서 자식이 없는 리프 노드만이 트리의 지름을 형성할 수 있다

`-` 리프 노드의 부모 노드를 고려하자

`-` 해당 부모 노드는 여러개의 리프 노드를 가진다

`-` 여러 리프 노드 중 부모와 연결된 간선의 가중치가 더 큰 리프 노드만 트리의 지름을 형성할 수 있다

`-` 부모 노드 $p$와의 가중치가 $w_1$인 $s_1$ 리프 노드가 다른 리프 노드 $u$와 트리의 지름을 형성한다고 해보자

`-` 해당 부모 노드의 또 다른 리프 노드 $s_2$는 $w_1$보다 큰 $w_2$의 가중치를 가진다고 하자

`-` 트리의 지름 $s_1 \to p \to u$보다 $s_2\to p \to u$가 더 큰 지름을 형성한다

`-` 따라서 여러 리프 노드 중 부모 노드와 연결된 간선의 가중치가 더 큰 리프 노드만 탐색하면 된다

`-` 단, 예외가 있는데 하나의 부모에 있는 두 리프 노드가 트리의 지름을 형성하면 위의 이론을 적용하면 안된다

`-` 이를 위해 부모와 연결된 리프 노드 중 간선의 가중치가 큰 순서대로 $2$개를 골라 더한 값을 전역 트리의 지름으로 갱신 시도해야 한다

`-` 아무튼 트리의 지름은 부모와 연결된 리프 노드를 잇는 선의 길이이다

`-` 그런데 이를 재귀적으로 반복할 수 있다

`-` 부모 노드 $p$와 리프 노드 $s_1, s_2,s_3$를 고려하자 (간선의 가중치가 큰 순서는 $s_1, s_2,s_3$라 하자) 

`-` 리프 노드 $s_1$이 선택됐다면 경로는 $p \to s_1$으로 고정되며 부모 노드 $p$는 사실상 리프 노드가 된다

`-` 트리의 지름은 $s_1 \to p \to s_2$와 $s_1\to p$를 지나는 다른 경로 중 더 큰 값으로 결정된다

`-` 초기에 모든 리프 노드에 대해 위의 작업을 수행하면 새로운 리프 노드가 생기고 이에 대해 반복하면 된다

- 알고리즘 요약

`1.` 루트 노드를 입력으로 넣는다

`2.` 입력된 노드의 자식 노드에 대해 함수를 재귀적으로 사용함

`3.` 함수 내에서 선택된 자식 노드와의 간선 가중치와 가장 큰 두 자식 노드를 잇는 트리의 지름을 기록함

`4.` 가장 큰 두 자식 노드를 잇는 트리의 지름을 트리의 지름 최댓값으로 갱신 시도

`5.` 선택된 자식 노드와의 간선 가중치를 기록하는 배열에 자식 노드 중 선택된 것을 추가로 더해준다

In [8]:
from collections import defaultdict


def dfs(start, tree, weight, child_weight):
    global MAX_DIAMETER
    edge_weights = []
    max_subtree_weight = 0
    for next_node in tree[start]:
        dfs(next_node, tree, weight, child_weight)
        w = weight[(start, next_node)] + child_weight[next_node]
        edge_weights.append(w)
        if w > max_subtree_weight:
            max_subtree_weight = w
    if len(edge_weights) > 2:
        edge_weights.sort(reverse=True)
        best_diameter = sum(edge_weights[:2])
    else:
        best_diameter = sum(edge_weights)
    MAX_DIAMETER = max(MAX_DIAMETER, best_diameter)
    child_weight[start] += max_subtree_weight


def solution():
    global MAX_DIAMETER
    n = int(input())
    tree = defaultdict(list)
    ROOT_NODE = 1
    MAX_DIAMETER = 0
    weight = {}
    child_weight = [0 for _ in range(n + 1)]
    for _ in range(n - 1):
        p, c, w = map(int, input().split())
        tree[p].append(c)
        weight[(p, c)] = w
    dfs(ROOT_NODE, tree, weight, child_weight)
    print(MAX_DIAMETER)


solution()

# input
# 12
# 1 2 3
# 1 3 2
# 2 4 5
# 3 5 11
# 3 6 9
# 4 7 1
# 4 8 7
# 5 9 15
# 5 10 4
# 6 11 6
# 6 12 10

 12
 1 2 3
 1 3 2
 2 4 5
 3 5 11
 3 6 9
 4 7 1
 4 8 7
 5 9 15
 5 10 4
 6 11 6
 6 12 10


45


`-` 참고로 트리의 지름을 구하는 또 다른 방법이 존재한다

`-` 임의의 한 노드에서 DFS를 사용해 가장 멀리 있는 노드를 찾고 해당 노드에서 DFS를 사용하면 트리의 지름을 구할 수 있다

## 트리의 지름

- 문제 출차: [백준 1167번](https://www.acmicpc.net/problem/1167)

`-` [트리의 지름](https://www.acmicpc.net/problem/1967) 문제의 심화 버전이다

`-` 나는 위의 문제를 이미 $O(N)$ 시간에 해결했기 때문에 비슷한 방법으로 이 문제도 풀 수 있다

`-` 차이점은 루트 노드가 주어지지 않고 부모 노드와 자식 노드 순서로 주어지지 않는 점이다

`-` 서로 연결된 노드는 서로의 인접 노드에 추가해야 한다

`-` 그리고 중복 방문을 허용하지 않는 코드를 DFS에 추가해야 한다

`-` 기존의 트리는 중복 방문을 할 일이 없었지만 지금은 그렇지 않다

`-` 기존 방식 대신 트리의 지름을 구하는 또 다른 방식을 사용해서 풀어볼까 한다

`-` 임의의 한 노드에서 DFS를 사용해 가장 멀리 있는 노드를 찾고 해당 노드에서 DFS를 사용하면 트리의 지름을 구할 수 있다

`-` 시간 복잡도는 $O(N)$으로 같다

`-` 위 방식이 성립하는 것을 증명해보자

- 알고리즘 증명

`-` $r$에서 DFS를 사용해 도착한 노드가 $u$라고 해보자

`-` 그리고 트리의 지름 양 끝 노드가 $x$와 $y$라고 해보자

`-` 트리의 노드는 서로 연결되어 있으며 두 노드를 잇는 경로는 하나만 존재한다

`-` 그러므로 만약 $r$이 $x$ 또는 $y$라면 자명하게 $u$는 $y$ 또는 $x$이다

`-` 이제 $r$이 $x$와 $y$ 둘 다 아니라고 해보자

`-` $u$가 $x$ 또는 $y$이면 당연히 성립하니 귀류법을 사용하고자 $u$가 전혀 다른 노드라고 해보자

`-` 그럼 $r \to u$ 경로가 있고 $x \to y$ 경로가 있을텐데 트리의 노드는 서로 연결되어 있다 했으므로 둘을 잇는 경로가 존재한다

`-` 배중률에 따라 두 경로를 모두 지나는 노드 $t$가 존재하거나 존재하지 않는다

`1.` 첫 번재로 $t$가 존재한다고 해보자

`-` 그러면 $r\to  t\to u$이고 $x\to t\to y$이다

`-` $r$에서 $u$가 가장 멀리 떨어져 있으므로 $t\to u$가 $t\to y$보다 길다

`-` 그렇지 않다면 $r$에서 가장 멀리 떨어진 노드는 $u$가 아니라 $y$가 되어야 한다

`-` 그런데 $t\to u$가 더 길기 때문에 $x\to t \to y$보다 $x\to t \to u$가 더 길다

`-` 즉, $u$는 $y$가 되어야 한다, 그렇지 않으면 모순이다

`2.` 두 번재로 $t$가 존재하지 않는다고 해보자

`-` 그러면 $r \to a \to u$이고 $x \to b \to y$이며 $a$와 $b$가 연결되어 있다고 해보자 (계속 언급했듯이 트리이므로 연결되어야만 한다)

`-` $u$가 $r$에서 가장 멀리 떨어졌으므로 $r \to a\to u$가 $r\to a\to b\to y$보다 길다

`-` 그런데 이게 사실이라면 트리의 지름은 $x\to b\to y$가 아니라 $x\to b \to a\to u$여야 하는데 이는 모순이다

`-` 따라서 이런한 경우는 존재할 수 없다

`-` 결과적으로 임의의 한 노드에서 가장 멀리 떨어진 노드는 트리의 지름의 양 끝 노드 중 하나이다

In [31]:
from collections import defaultdict


def dfs(node, graph, diameter=0, visited={}):
    global LAST_NODE, MAX_DIAMETER
    visited[node] = True
    for neighbor, weight in graph[node]:
        if neighbor not in visited:
            dfs(neighbor, graph, diameter + weight, visited)
    if diameter > MAX_DIAMETER:
        LAST_NODE = node  # 처음에 dfs에 입력한 노드와 가장 멀리 떨어진 노드
        MAX_DIAMETER = diameter


def solution():
    global LAST_NODE, MAX_DIAMETER
    V = int(input())
    graph = defaultdict(list)
    LAST_NODE = 1
    MAX_DIAMETER = 0
    start_node = 1
    for _ in range(V):
        u, *data, _ = map(int, input().split())
        for i in range(len(data) // 2):
            v, w = data[i * 2], data[i * 2 + 1]
            graph[u].append((v, w))
    dfs(start_node, graph, diameter=0, visited={})  # 트리의 지름을 잇는 두 점 중 하나를 탐색
    dfs(LAST_NODE, graph, diameter=0, visited={})  # 트리의 지름의 한 끝 점에서 나머지 끝 점을 탐색
    print(MAX_DIAMETER)


solution()

# input
# 5
# 1 3 2 -1
# 2 4 4 -1
# 3 1 2 4 3 -1
# 4 2 4 3 3 5 6 -1
# 5 4 6 -1

 5
 1 3 2 -1
 2 4 4 -1
 3 1 2 4 3 -1
 4 2 4 3 3 5 6 -1
 5 4 6 -1


11


`-` 주의 사항: dfs 함수의 visited 기본 값은 빈 딕셔너리로 되어 있다

`-` 인자의 기본 값으로 빈 리스트를 사용했을 때와 마찬가지의 부작용을 가지고 있으니 주의해야 한다

`-` visited 인자를 비어두고 함수를 다시 실행하면 전의 함수에서 갱신된 visited가 그대로 있다

## 트리

- 문제 출처: [백준 1068번](https://www.acmicpc.net/problem/1068)

`-` 노드를 클래스로 구현한 후 부모 노드와 자식 노드를 멤버 변수로 가지자

`-` 임의의 노드를 삭제하면 부모 노드로 올라가서 해당 노드를 삭제하자

`-` 또한 해당 노드의 자식 노드에 대해 재귀적으로 반복하자

`-` 그리고 삭제한 노드에 대해 삭제한 표식을 부여하자

`-` 자식 노드가 하나도 없다면 리프 노드인 것이다

`-` 노드 삭제 후 모든 노드를 순회하면서 리프 노드를 카운팅하면 정답이다

In [24]:
class Node:
    def __init__(self, x):
        self.x = x  # 자기 자신 번호
        self.left_node = None
        self.right_node = None
        self.parent = None
        self.is_deleted = False

    def set_parent(self, parent_node):
        self.parent = parent_node

    def set_child(self, child_node):
        if self.left_node is None:
            self.left_node = child_node
        elif self.right_node is None:
            self.right_node = child_node

    def remove_child(self, child):
        if self.left_node is not None and self.left_node.x == child.x:
            self.left_node = None
        elif self.right_node is not None and self.right_node.x == child.x:
            self.right_node = None

    def delete(self):
        self.is_deleted = True
        if self.parent is not None:
            self.parent.remove_child(self)
        if self.left_node is not None:
            self.left_node.delete()
        if self.right_node is not None:
            self.right_node.delete()

    def is_leaf_node(self):
        return self.left_node is None and self.right_node is None


def solution():
    N = int(input())
    NONE = -1
    nodes = {i: Node(i) for i in range(N)}
    parents = list(map(int, input().split()))
    node_to_delete = int(input())
    for i, p in enumerate(parents):
        if p != NONE:
            nodes[i].set_parent(nodes[p])
            nodes[p].set_child(nodes[i])
    nodes[node_to_delete].delete()
    answer = 0
    for i in range(N):
        if not nodes[i].is_deleted and nodes[i].is_leaf_node():
            answer += 1
    print(answer)


solution()

# input
# 9
# -1 0 0 2 2 4 4 6 6
# 4

 9
 -1 0 0 2 2 4 4 6 6
 4


2


## 이진 검색 트리

- 문제 출처: [백준 5639번](https://www.acmicpc.net/problem/5639)

`-` 이진 검색 트리의 전위 순회 결과를 바탕으로 트리를 재구성하고 후위 순회 결과를 출력하면 된다

`-` 이진 검색 트리에서 노드의 왼쪽 서브 트리의 모든 노드의 키는 노드의 키보다 작고 오른쪽 서브 트리의 모든 노드의 키는 노드의 키보다 크다

`-` 루트 노드를 초기값으로 가지고 시작하자

`-` 다음 입력된 노드의 키 값이 현재 노드의 키 값보다 작으면 왼쪽 자식 노드로 이동한다

`-` 크다면 오른쪽 자식 노드로 이동한다

`-` 이를 이동할 자식 노드가 없을 때까지 재귀적으로 반복하며 종료 조건에 도달하면 입력된 노드를 해당 자식 노드로 설정한다

`-` 위 알고리즘의 시간 복잡도를 계산해보자

`-` 최악의 경우 트리가 오른쪽으로만 뻗어 나갈 수 있다 (트리의 높이가 $N$이다)

`-` 이 경우 루트 노드를 제외한 $N-1$번의 순회를 하며 순회마다 $1,2,\cdots,N-1$번의 비교를 진행한다

`-` 따라서 알고리즘의 시간 복잡도는 $O\left(N^2\right)$이며 노드의 개수 $N$은 최대 $10000$이므로 시간 제한에 걸리지 않는다

`-` 위와 같이 하면 전위 순회 결과를 바탕으로 이진 검색 트리를 재구성할 수 있다

`-` 후위 순회는 [트리 순회](https://www.acmicpc.net/problem/1991) 문제에서 했듯이 하면 된다

In [55]:
import sys

sys.setrecursionlimit(10**5)


class Node:
    def __init__(self, key):
        self.key = key  # 자기 자신 번호
        self.left_child = None
        self.right_child = None


def postorder_traversal(root_node):
    # 왼쪽 -> 오른쪽 -> 루트
    if root_node.left_child is not None:
        postorder_traversal(root_node.left_child)
    if root_node.right_child is not None:
        postorder_traversal(root_node.right_child)
    print(root_node.key)

    
def update_tree(current_node, next_node):
    if current_node.key < next_node.key:
        if current_node.right_child is None:
            current_node.right_child = next_node
            return
        update_tree(current_node.right_child, next_node)
    else:
        if current_node.left_child is None:
            current_node.left_child = next_node
            return
        update_tree(current_node.left_child, next_node)
    

def make_tree_from_preorder_traversal(root_node):
    test_key = -1
    while True:
        try:
            key = int(input())
            if key == test_key:
                break
        except EOFError:
            break
        next_node = Node(key)
        update_tree(root_node, next_node)
    

def solution():
    root_node = Node(int(input()))
    make_tree_from_preorder_traversal(root_node)
    postorder_traversal(root_node)


solution()

# input
# 50
# 30
# 24
# 5
# 28
# 45
# 98
# 52
# 60
# -1  # For test

 50
 30
 24
 5
 28
 45
 98
 52
 60
 -1


5
28
24
45
30
60
52
98
50


## 트리와 쿼리

- 문제 출처: [백준 15681번](https://www.acmicpc.net/problem/15681)

`-` 부모를 루트로 하는 서브 트리에 속한 정점의 수는 자식들을 루트로 하는 서브 트리에 속한 정점의 수들의 합에 $1$을 더한 것과 같다

`-` 부모 정점 $P$와 자식 정점 $X, Y$가 있다고 해보자

`-` 임의의 노드 $u$를 루트로 하는 서브 트리에 속한 정점의 수를 $f(u)$라고 하자

`-` 그럼 $f(P) = f(X) + f(Y) + 1$이다 ($1$을 더하는 건 부모 자기 자신이다)

`-` 트리 구조를 생각하면 위의 식은 자연스럽게 성립하니 이를 바탕으로 문제를 해결하자

`-` 리트 노드라면 함숫값은 $1$이다 (리트 노드를 루트로 하는 서브 트리에 속한 정점은 자기 자신밖에 없다)

In [10]:
import sys
from collections import defaultdict

sys.setrecursionlimit(10**5)


class Node:
    def __init__(self, key):
        self.key = key
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)


def make_tree(root_node, graph):
    root_node = Node(root_node)
    stack = [root_node]
    visited = {root_node.key: True}
    while stack:
        node = stack.pop()
        for neighbor in graph[node.key]:
            if neighbor not in visited:
                neighbor = Node(neighbor)
                node.add_child(neighbor)
                visited[neighbor.key] = True
                stack.append(neighbor)
    return root_node  # 루트 노드에 tree의 정보가 담겨있다


def count_nodes_in_subtree(root_node):
    if root_node.key in DP:
        return DP[root_node.key]
    num_nodes = 1  # 자기 자신
    for child in root_node.children:
        num_nodes += count_nodes_in_subtree(child)
    DP[root_node.key] = num_nodes
    return num_nodes


def solution():
    global DP
    N, R, Q = map(int, input().split())
    graph = defaultdict(list)
    DP = {}
    for _ in range(N - 1):
        U, V = map(int, input().split())
        graph[U].append(V)
        graph[V].append(U)
    root_node = make_tree(R, graph)
    count_nodes_in_subtree(root_node)
    for _ in range(Q):
        U = int(input())
        print(DP[U])


solution()

# input
# 9 5 3
# 1 3
# 4 3
# 5 4
# 5 6
# 6 7
# 2 3
# 9 6
# 6 8
# 5
# 4
# 8

 9 5 3
 1 3
 4 3
 5 4
 5 6
 6 7
 2 3
 9 6
 6 8
 5


9


 4


4


 8


1


- 힌트에 설명된 코드 (chatgpt가 작성함)

`-` 불필요한 클래스 객체 필요 없이 한 번에 탐색으로 모든 쿼리에 대답할 수 있는 코드

`-` 깔끔하게 구현되었다 (한 번의 탐색으로 모든 쿼리에 대답할 수 있다)

`-` 양방향 그래프 구조에서 루트에서 내려가는 단방향 트리처럼 탐색을 어떻게 하나 싶었는데 자식이 부모와 다른 경우만 탐색하도록 구현했다

`-` 그래서 부모로 거슬러 올라가지 않고 자식만 방문할 수 있다

```python
import sys
from collections import defaultdict

sys.setrecursionlimit(10**5)


def dfs(node, parent):
    subtree_size[node] = 1  # 자기 자신 포함
    for child in tree[node]:
        if child == parent:
            continue
        subtree_size[node] += dfs(child, node)
    return subtree_size[node]


def solution():
    global tree, subtree_size
    N, R, Q = map(int, sys.stdin.readline().split())
    tree = defaultdict(list)
    subtree_size = {}
    for _ in range(N - 1):
        U, V = map(int, sys.stdin.readline().split())
        tree[U].append(V)
        tree[V].append(U)
    dfs(R, -1)  # 루트 R에서 시작하는 DFS
    for _ in range(Q):
        U = int(sys.stdin.readline())
        print(subtree_size[U])


solution()
```

## 우수 마을

- 문제 출처: [백준 1949번](https://www.acmicpc.net/problem/1949)

`-` $p$를 루트로 하는 서브 트리에 있는 우수 마을의 총 주민수를 고려하자

`-` 이때 $p$가 우수 마을이거나 $p$의 자식이 우수 마을이다

`-` 2번 조건에 의해 둘 다 우수 마을일 수 없고 3번 조건에 의해 둘 다 우수 마을이 아닐 수는 없다

`-` 2번 조건과 3번 조건을 충족하기 위해 $p$가 우수 마을인지와 $p$의 부모가 우수 마을인지를 한 번에 고려할 것이다 

`-` `dp[p][1][0]`을 $p$의 부모는 우수 마을이 아니고 $p$가 우수 마을일 때 $p$를 루트로 하는 서브 트리에 있는 우수 마을의 총 주민수라고 하자

`-` 배열의 $2$번째 값은 $p$가 우수 마을인지 아닌지를 뜻하며 배열의 $3$번째 값은 $p$의 부모가 우수 마을인지 아닌지를 뜻한다 

`-` $p$의 자식들을 $c_1,c_2,\cdots, c_n$라고 하자

`-` 그럼 아래의 점화식이 성립한다

`-` $\operatorname{dp}[p][1][0] = \operatorname{village}[p] + \operatorname{dp}[c_1][0][1] + \operatorname{dp}[c_2][0][1] + \cdots + \operatorname{dp}[c_n][0][1]$이다

`-` $\operatorname{dp}[p][0][1] = \max(\{\operatorname{dp}[c_1][s_1][0] + \operatorname{dp}[c_2][s_2][0] + \cdots + \operatorname{dp}[c_n][s_n][0],\cdots\})$이다 (단, $s_i$는 $0$ 또는 $1$)

`-` $\operatorname{dp}[p][0][0] = \max(\{\operatorname{dp}[c_1][s_1][0] + \operatorname{dp}[c_2][s_2][0] + \cdots + \operatorname{dp}[c_n][s_n][0],\cdots\})$이다 (단, $s_i$중 적어도 하나는 $1$)

`-` 점화식 $2$번째에선 각각의 최댓값을 선택하면 된다

`-` 점화식 $3$번째에선 먼저 각각의 최댓값을 선택한다

`-` 만약 $s_i$가 모두 $0$이라면 $s_i$를 $1$로 바꿨을 때 손실이 가장 적은 것을 $1$로 바꾼다

`-` 위의 점화식을 기반으로 배열을 채워갈 때 불가능한 경우는 제외해야 한다

`-` 리프 노드 $l$에 대해 종료 조건을 정의하자

`-` $\operatorname{dp}[l][1][0] = \operatorname{village}[l]$

`-` $\operatorname{dp}[l][0][1] = 0$

`-` $\operatorname{dp}[l][0][0] = 0$, 불가능한 경우이지만 형식을 맞추기 위함 (어차피 최댓값이 될 수 없어서 괜찮다)

`-` `village[x]`는 마을 $x$의 주민 수이다

In [3]:
import sys
from collections import defaultdict

sys.setrecursionlimit(10**5)


def dfs(tree, node, parent):
    dp[node][1][0] += villages[node]
    excellent_at_least = False
    min_gap = INF
    for child in tree[node]:
        if child == parent:
            continue
        dp1, dp2, dp3 = dfs(tree, child, node)
        if dp1 >= dp3:
            dp_max = dp1
            excellent_at_least = True
        else:
            dp_max = dp3
            dp_min = dp1
            min_gap = min(min_gap, dp_max - dp_min)
        dp[node][1][0] += dp2
        dp[node][0][1] += dp_max
        dp[node][0][0] += dp_max
    if min_gap == INF:
        min_gap = 0
    if not excellent_at_least:
        dp[node][0][0] -= min_gap
    return dp[node][1][0], dp[node][0][1], dp[node][0][0]


def solution():
    global villages, INF, dp
    N = int(input())
    villages = [0] + list(map(int, input().split()))
    tree = defaultdict(list)
    INF = float("inf")
    for _ in range(N - 1):
        u, v = map(int, input().split())
        tree[u].append(v)
        tree[v].append(u)
    dp = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(N + 1)]  # dp[x][1][0]은 루트가 x, x는 우수, x의 부모는 우수 아님
    ROOT = 1
    NONE = -1
    dp1, dp2, dp3 = dfs(tree, ROOT, NONE)
    answer = max(dp1, dp2, dp3)
    print(answer)


solution()

# input
# 7
# 1000 3000 4000 1000 2000 2000 7000
# 1 2
# 2 3
# 4 3
# 4 5
# 6 2
# 6 7

 7
 1000 3000 4000 1000 2000 2000 7000
 1 2
 2 3
 4 3
 4 5
 6 2
 6 7


14000


`-` 다른 사람 코드 보고 생각남

`-` 생각해보니 $\operatorname{dp}[p][0][0]$은 고려할 필요가 없다

`-` 항상 $\operatorname{dp}[p][0][1]$이 더 크다

## 트리

- 문제 출처: [백준 4803번](https://www.acmicpc.net/problem/4803)

`-` 일단 dfs를 통해 연결된 정점끼리 그룹 짓자

`-` 각 그룹에 대해 모든 정점이 연결되어 있으므로 사이클이 없다면 트리이다

`-` 그러기 위해선 간선의 개수가 $n-1$개여야 한다

In [8]:
from collections import defaultdict


def dfs(start, graph, visited):
    visited.add(start)
    for neighbor in graph[start]:
        if neighbor not in visited:
            dfs(neighbor, graph, visited)


def is_tree(group, graph):
    n = len(group)
    num_edges = 0
    for u in group:
        num_edges += len(graph[u])
    num_edges /= 2
    return num_edges == n - 1


def solve_testcase(ith):
    n, m = map(int, input().split())
    if n == 0 and m == 0:
        return end_key
    groups = []
    graph = defaultdict(list)
    for _ in range(m):
        u, v = map(int, input().split())
        graph[u].append(v)
        graph[v].append(u)
    all_visited = set()
    visited = set()
    for node in range(1, n + 1):
        if node in all_visited:
            continue
        dfs(node, graph, visited)
        groups.append(list(visited))
        all_visited = all_visited.union(visited)
        visited.clear()
    num_trees = 0
    for group in groups:
        if is_tree(group, graph):
            num_trees += 1
    if num_trees == 0:
        print(f"Case {ith}: No trees.")
        return
    if num_trees == 1:
        print(f"Case {ith}: There is one tree.")
        return
    print(f"Case {ith}: A forest of {num_trees} trees.")


def solution():
    global end_key
    ith = 1
    end_key = -1
    while True:
        result = solve_testcase(ith)
        ith += 1
        if result == end_key:
            break


solution()

# input
# 6 3
# 1 2
# 2 3
# 3 4
# 6 5
# 1 2
# 2 3
# 3 4
# 4 5
# 5 6
# 6 6
# 1 2
# 2 3
# 1 3
# 4 5
# 5 6
# 6 4
# 0 0

 6 3
 1 2
 2 3
 3 4


Case 1: A forest of 3 trees.


 6 5
 1 2
 2 3
 3 4
 4 5
 5 6


Case 2: There is one tree.


 6 6
 1 2
 2 3
 1 3
 4 5
 5 6
 6 4


Case 3: No trees.


 0 0


## 트리의 순회

- 문제 출처: [백준 2263번](https://www.acmicpc.net/problem/2263)

`-` 중위 순회와 후위 순회가 주어졌을 때 전위 순회를 찾으면 되는 문제이다

`-` 노드가 $7$개이고 높이가 $2$인 완전 이진 트리의 전위 순회, 중위 순회, 후위 순회를 생각해보자

`-` 전위 순회: `1 -> 2 -> 4 -> 5 -> 3 -> 6 -> 7`

`-` 중위 순회: `4 -> 2 -> 5 -> 1 -> 6 -> 3 -> 7`

`-` 후위 순회: `4 -> 5 -> 2 -> 6 -> 7 -> 3 -> 1`

`-` 전위 순회는 `root -> left -> right` 순으로 탐색하므로 루트 노드의 위치를 알아야 한다

`-` 후위 순회는 루트 노드를 마지막으로 탐색하기 때문에 마지막으로 탐색한 노드가 루트 노드이다

`-` 즉, 위의 예시에선 `1`이 루트 노드이다

`-` 중위 순회는 `left -> root -> right`이므로 루트 노드 이전은 왼쪽 서브 트리이고 이후는 오른쪽 서브 트리이다

`-` 즉, `4 -> 2 -> 5`는 왼쪽 서브 트리이고 `1`은 루트 노드이고 `3 -> 6 -> 7`은 오른쪽 서브 트리이다

`-` 전위 순회는 루트 노드를 먼저 출력하므로 중위 순회와 후위 순회로부터 찾은 루트 노드를 출력하면 된다

`-` 왼쪽 서브 트리와 오른쪽 서브 트리에 대해 위의 작업을 반복하며 서브 트리의 루트 노드를 출력해 나가면 전위 순회를 찾을 수 있다

`-` 중위 순회에서 루트 노드인 `1`의 다음 순회 노드인 `6`을 기준으로 후위 순회에서 `6` 이전은 왼쪽 서브 트리, 이후는 오른쪽 서브 트리이다

`-` 그럼 중위 순회에서 `4 -> 2 -> 5`만 고려하고 후위 순회에서 `4 -> 5 -> 2`만 고려하여 루트 노드를 출력하면 되고 이는 위의 방법과 동일하게 수행하면 된다

`-` 이를 재귀적으로 수행하면 함수 실행 한 번에 루트 노드 하나를 출력하므로 $O(N)$에 전위 순회를 찾을 수 있다

`-` 이를 수행하기 위해 다음과 같은 데이터를 가지고 있어야 한다

`-` 우선 중위 순회와 후위 순회의 배열을 가지고 있어야 하며 또한 특정 노드의 탐색 순서를 알아야 하므로 딕셔너리로 노드와 탐색 순서를 키와 값으로 하여 관리해야 한다

`-` 중위 순회에선 서브 트리의 루트 노드의 탐색 순서를 알아야 하며 후위 순회에선 중외 순회에서 찾은 서브 트리의 루트 노드 다음 노드의 탐색 순서를 알야아 한다

`-` 틀려서 반례 참고 했다

`-` 내가 완전 이진 트리에 대해 잘못 알고 있었다 ($2^h-1$개 꼴만 완전 이진 트리라고 생각했음)

`-` 입력으로 완전 이진 트리가 들어올 수도 있고 아닐 수도 있다 

`-` 즉, 한쪽으로 편향된 트리일 수도 있다

`-` 이러면 재귀 깊이가 $O(\log N)$이 아니라 $O(N)$이므로 재귀 깊이를 늘려줘야 한다

`-` 나는 완전 이진 트리를 가정하고 루트 노드를 기준으로 왼쪽 서브 트리와 오른쪽 서브 트리에 대해 재귀적으로 적용했는데 한쪽의 서브 트리만 존재할 수 있다

`-` 루트 노드가 고려하고 있는 중위 순회 배열에서 왼쪽 끝이나 오른쪽 끝에 존재하면 한쪽의 서브 트리만 가지고 있는 것이다

`-` 한쪽의 서브 트리만 가지고 있으면 함수를 두 번이 아닌 한 번만 실행해야 한다

`-` 원래 함수에선 후위 순회 배열의 `left, right` 인덱스만 고려했는데 중위 순회의 `left, right` 인덱스도 같이 고려하여 서브 트리가 어떻게 나뉘는지 판단하자

In [22]:
import sys

sys.setrecursionlimit(10**5 + 2)


def solution():
    global inorder, postorder, inorder_dict, postorder_dict, preorder
    n = int(input())
    inorder = list(map(int, input().split()))
    postorder = list(map(int, input().split()))
    inorder_dict = {node: turn for turn, node in enumerate(inorder)}
    postorder_dict = {node: turn for turn, node in enumerate(postorder)}
    preorder = []
    find_preorder(0, n - 1, 0, n - 1)
    print(*preorder)


def find_preorder(left_post, right_post, left_in, right_in):
    if right_post == left_post:
        preorder.append(postorder[left_post])
        return
    root = postorder[right_post]
    preorder.append(root)
    root_index = inorder_dict[root]
    if root_index == left_in:
        find_preorder(left_post, right_post - 1, left_in + 1, right_in)
        return
    if root_index == right_in:
        find_preorder(left_post, right_post - 1, left_in, right_in - 1)
        return
    right_start = inorder[root_index + 1]
    right_start_index = postorder_dict[right_start]
    find_preorder(left_post, right_start_index - 1, left_in, root_index - 1)
    find_preorder(right_start_index, right_post - 1, root_index + 1, right_in)


solution()

# input
# 7
# 4 2 5 1 6 3 7
# 4 5 2 6 7 3 1

 7
 4 2 5 1 6 3 7
 4 5 2 6 7 3 1


1 2 4 5 3 6 7


## 사회망 서비스(SNS)

- 문제 출처: [백준 2533번](https://www.acmicpc.net/problem/2533)

`-` 어떤 노드가 얼리 아답터인지 아닌지는 그 노드의 부모와 자식이 얼리 아답터인지 아닌지에 영향을 받는다

`-` 리프 노드는 자식을 가지지 않으므로 부모의 얼리 아답터 유무만 판단해도 된다

`-` 어떤 노드를 루트로 하는 서브 트리에 속한 최소 얼리 아답터의 수를 고려하자

`-` `dp[u][1][0]`를 노드 $u$가 얼리 아답터이고 $u$의 부모가 얼리 아답터가 아닐 때 $u$가 루트인 서브 트리에 속한 최소 얼리 아답터의 수라고 하자

`-` $u$가 얼리 아답터라면 부모와 자식은 얼리 아답터이든 아니든 상관 없다

`-` $u$가 얼리 아답터가 아니라면 부모와 자식은 얼리 아답터여야 한다

`-` 노드 $u$의 자식들을 $c_1, c_2, \cdots, c_u$라고 할 때 다음의 점화식이 성립한다

`-` $\operatorname{dp}[u][1][0] = \operatorname{dp}[u][1][1] = \min(\operatorname{dp}[c_1][0][1], \operatorname{dp}[c_1][1][1]) + \min(\operatorname{dp}[c_2][0][1], \operatorname{dp}[c_2][1][1]) + \cdots +  + \min(\operatorname{dp}[c_u][0][1], \operatorname{dp}[c_u][1][1])$

`-` $\operatorname{dp}[u][0][1] = \operatorname{dp}[c_1][1][0] +\operatorname{dp}[c_2][1][0]  + \cdots + \operatorname{dp}[c_u][1][0]$

`-` 또한, 리프 노드 $l$에 대해 $\operatorname{dp}[l][0][1] = 0,\; \operatorname{dp}[l][1][0] = \operatorname{dp}[l][1][1] = 1$이다

`-` 결과적으로 $\min(\operatorname{dp}[u][0][1], \operatorname{dp}[u][1][0], \operatorname{dp}[u][1][1])$이 정답이 된다

In [3]:
import sys
from collections import defaultdict

sys.setrecursionlimit(10**6)


def dfs(tree, node, parent):
    dp[node][0][1] = 0
    dp[node][1][0] = 1
    dp[node][1][1] = 1
    for child in tree[node]:
        if child == parent:
            continue
        dfs(tree, child, node)
        dp[node][0][1] += dp[child][1][0]
        dp[node][1][0] += min(dp[child][0][1], dp[child][1][1])
        dp[node][1][1] += min(dp[child][0][1], dp[child][1][1])


def solution():
    global dp
    N = int(input())
    tree = defaultdict(list)
    for _ in range(N - 1):
        u, v = map(int, input().split())
        tree[u].append(v)
        tree[v].append(u)
    dp = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(N + 1)]
    ROOT_NODE = 1
    NONE = -1
    dfs(tree, ROOT_NODE, NONE)
    answer = min(dp[ROOT_NODE][0][1], dp[ROOT_NODE][1][0], dp[ROOT_NODE][1][1])
    print(answer)


solution()

# input
# 9
# 1 2
# 1 3
# 2 4
# 3 5
# 3 6
# 4 7
# 4 8
# 4 9

 9
 1 2
 1 3
 2 4
 3 5
 3 6
 4 7
 4 8
 4 9


3


## 트리의 독립집합

- 문제 출처: [백준 2213번](https://www.acmicpc.net/problem/2213)

`-` [우수 마을](https://www.acmicpc.net/problem/1949) 문제에 역추적을 더한 문제이다

`-` 여태까지 트리 DP 문제를 잘 해결했다면 이 문제도 쉽게 풀 수 있다

In [5]:
import sys
from collections import defaultdict


def dfs(root, parent, graph, weights):
    for child in graph[root]:
        if child == parent:
            continue
        dfs(child, root, graph, weights)
        if dp[child][1] > dp[child][0]:
            dp[root][0] += dp[child][1]
            track[root][0].append((child, 1))
        else:
            dp[root][0] += dp[child][0]
            track[root][0].append((child, 0))
        dp[root][1] += dp[child][0]
        track[root][1].append((child, 0))


def solution():
    global dp, track
    n = int(input())
    weights = [0] + list(map(int, input().split()))
    dp = [[0 for _ in range(2)] for _ in range(n + 1)]  # dp[x][0]을 x가 루트인 서브 트리에서 x를 제외한 최대 독립 집합이라고 하자 (dp[x][1]은 포함)
    track = [[[] for _ in range(2)] for _ in range(n + 1)]  # track[x][0]은 x가 루트인 서브 트리에서 x는 사용 안할 때 최대 독립 집합에 포함되는 자식 노드
    for u in range(1, n + 1):  # 초깃값
        dp[u][0] = 0
        dp[u][1] = weights[u]
    graph = defaultdict(list)
    for _ in range(n - 1):
        u, v = map(int, input().split())
        graph[u].append(v)
        graph[v].append(u)
    ROOT = 1
    NONE = -1
    dfs(ROOT, NONE, graph, weights)
    answer = []
    if dp[ROOT][1] > dp[ROOT][0]:
        answer.append(ROOT)
        node, is_in = ROOT, 1
    else:
        node, is_in = ROOT, 0
    stack = [(node, is_in)]
    while stack:
        node, is_in = stack.pop()
        for child, is_in in track[node][is_in]:
            if is_in:
                answer.append(child)
            stack.append((child, is_in))
    answer.sort()
    print(max(dp[ROOT]))
    print(*answer)


solution()

# input
# 7
# 10 30 40 10 20 20 70
# 1 2
# 2 3
# 4 3
# 4 5
# 6 2
# 6 7

 7
 10 30 40 10 20 20 70
 1 2
 2 3
 4 3
 4 5
 6 2
 6 7


140
1 3 5 7
